## Load 'jupyter notebook' from Conda Terminal before beginning to use GPU

In [5]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Sequential
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.optimizers import SGD
import numpy as np
import tensorflow as tf

# Testing to make sure TensorFlow GPU is working
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print('Tensorflow: ', tf.__version__)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1646846974717347302
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3649437696
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7751581222681017358
physical_device_desc: "device: 0, name: GeForce GTX 1050, pci bus id: 0000:01:00.0, compute capability: 6.1"
]
Tensorflow:  1.11.0


### Load data, shuffle data, normalize data, split data into training and validation

In [6]:
from sklearn.utils import shuffle

# Load the data
X_train = np.load("X_train_3.dat")
y_train = np.load("y_train_3.dat")


X_train, y_train = shuffle(X_train, y_train, random_state=42)
# reshape so in form for CNN-Keras
#X_train = X_train.reshape(X_train.shape[0], 174, 124, 1)

# Normalize the data
X_train = X_train/255

print(X_train.shape)
print(y_train.shape)

# split data into validation set
training_set_size = int(X_train.shape[0] * .80)
X_train_validation = X_train[training_set_size:, :,:,:]
y_train_validation = y_train[training_set_size:, :]
y_train = y_train[:training_set_size,:]
X_train = X_train[:training_set_size, :, :, :]
print("Validation x train set:" + str(X_train_validation.shape))
print("X train set:" + str(X_train.shape))
print("Validation y train set:" + str(y_train_validation.shape))
print("Y train set:" + str(y_train.shape))

(9000, 124, 174, 3)
(9000, 10)
Validation x train set:(1800, 124, 174, 3)
X train set:(7200, 124, 174, 3)
Validation y train set:(1800, 10)
Y train set:(7200, 10)


## Build the CNN network

In [7]:
# VGG-16 like network from Andrew Ng's course on Coursera

model = Sequential()

# input: 100x100 images with 3 channels -> (100, 100, 3) tensors.
# this applies 32 convolution filters of size 3x3 each.
model.add(Conv2D(64, (7,7), strides=(1,1), input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(128, (11,11), strides=(1,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

# model.add(Conv2D(128, (11, 11), strides=(1,1)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

# model.add(Conv2D(256, (11, 11), strides=(1,1)))
# model.add(Activation('relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.5))

# # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

## Begin training the model

In [8]:
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint

mcp = ModelCheckpoint("models/best_model", monitor='val_acc', verbose=0, 
                      save_best_only=True, save_weights_only=False, mode='max', period=1)

adam = Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.001, amsgrad=False)
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(adam, 'categorical_crossentropy',metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=200 , validation_data=(X_train_validation, y_train_validation), 
         callbacks = [mcp])

Train on 7200 samples, validate on 1800 samples
Epoch 1/200
7200/7200 [==============================] - 72s 10ms/step - loss: 1.9674 - acc: 0.3543 - val_loss: 2.9018 - val_acc: 0.2189
Epoch 2/200
7200/7200 [==============================] - 72s 10ms/step - loss: 1.5448 - acc: 0.4722 - val_loss: 1.5530 - val_acc: 0.4439
Epoch 3/200
7200/7200 [==============================] - 70s 10ms/step - loss: 1.2682 - acc: 0.5629 - val_loss: 1.9701 - val_acc: 0.3561
Epoch 4/200
7200/7200 [==============================] - 70s 10ms/step - loss: 1.0859 - acc: 0.6215 - val_loss: 1.2020 - val_acc: 0.6000
Epoch 5/200
7200/7200 [==============================] - 70s 10ms/step - loss: 0.9114 - acc: 0.6939 - val_loss: 2.0327 - val_acc: 0.4139
Epoch 6/200
7200/7200 [==============================] - 70s 10ms/step - loss: 0.7885 - acc: 0.7300 - val_loss: 1.2204 - val_acc: 0.5833
Epoch 7/200
7200/7200 [==============================] - 70s 10ms/step - loss: 0.5969 - acc: 0.8024 - val_loss: 1.0107 - val_acc: 

7200/7200 [==============================] - 69s 10ms/step - loss: 0.0065 - acc: 0.9985 - val_loss: 1.0057 - val_acc: 0.7761
Epoch 61/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0076 - acc: 0.9986 - val_loss: 1.0067 - val_acc: 0.7767
Epoch 62/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0054 - acc: 0.9986 - val_loss: 1.0323 - val_acc: 0.7817
Epoch 63/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0060 - acc: 0.9985 - val_loss: 0.9947 - val_acc: 0.7883
Epoch 64/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0055 - acc: 0.9989 - val_loss: 1.0248 - val_acc: 0.7767
Epoch 65/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0080 - acc: 0.9982 - val_loss: 0.9848 - val_acc: 0.7783
Epoch 66/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0070 - acc: 0.9981 - val_loss: 0.9868 - val_acc: 0.7828
Epoch 67/200
7200/7200 [=======================

7200/7200 [==============================] - 69s 10ms/step - loss: 0.0025 - acc: 0.9992 - val_loss: 1.0289 - val_acc: 0.7900
Epoch 120/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0030 - acc: 0.9992 - val_loss: 1.0394 - val_acc: 0.7850
Epoch 121/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0036 - acc: 0.9989 - val_loss: 1.0438 - val_acc: 0.7861
Epoch 122/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0031 - acc: 0.9992 - val_loss: 1.0430 - val_acc: 0.7861
Epoch 123/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0029 - acc: 0.9992 - val_loss: 1.0306 - val_acc: 0.7839
Epoch 124/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0038 - acc: 0.9989 - val_loss: 1.0300 - val_acc: 0.7861
Epoch 125/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0018 - acc: 0.9999 - val_loss: 1.0280 - val_acc: 0.7906
Epoch 126/200
7200/7200 [================

7200/7200 [==============================] - 69s 10ms/step - loss: 0.0026 - acc: 0.9990 - val_loss: 1.0668 - val_acc: 0.7900
Epoch 179/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0023 - acc: 0.9994 - val_loss: 1.0569 - val_acc: 0.7939
Epoch 180/200
7200/7200 [==============================] - 70s 10ms/step - loss: 0.0015 - acc: 0.9993 - val_loss: 1.0559 - val_acc: 0.7956
Epoch 181/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0029 - acc: 0.9992 - val_loss: 1.0590 - val_acc: 0.7911
Epoch 182/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0028 - acc: 0.9989 - val_loss: 1.0394 - val_acc: 0.7906
Epoch 183/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0027 - acc: 0.9993 - val_loss: 1.0558 - val_acc: 0.7911
Epoch 184/200
7200/7200 [==============================] - 69s 10ms/step - loss: 0.0023 - acc: 0.9994 - val_loss: 1.0745 - val_acc: 0.7856
Epoch 185/200
7200/7200 [================

In [ ]:
model.save("models/trained_music_classifier.h5")

## Evaluate the model

In [57]:
score = model.evaluate(X_train_validation, y_train_validation, batch_size=32, verbose=1)
print(score)

50/50 [==============================] - 0s 1ms/step
[3.419465198516846, 0.5800000071525574]


### Reset Keras

In [53]:
from keras import backend as K
K.clear_session()